# HW5 Solution

# Problem 1: Support Vector Machines and Kernels

### **1a**: Describe a setting of the hyperparameters `gamma` and `C` that is sure to *overfit* to any typical training set and achieve zero training error.


To overfit, we should set `gamma` very large (e.g. $10^5$ or bigger). Setting `gamma` large makes the decision boundary as flexible as possible. 

Given a fixed training point $x$, any nearby point $x+ \Delta$ will be rated as *unsimilar* to $x$ because the kernel value is small. Let $||\Delta||_2^2 = \sum_{f} \Delta_f^2 = \epsilon$

\begin{align}
k(x, x+\Delta) = e^{-\gamma ||(x+ \Delta) - x||_2^2} 
= e^{-\gamma \epsilon} \quad \approx 0 ~~\text{if gamma is large}
\end{align}

Thus, if the kernel rates nearby points as dissimilar, the predicted value can vary widely and lead to flexible training boundaries and overfitting.

To overfit, we should also set `C` to a large value like $10^5$ or bigger. Remember that there are two terms here:

    margin-width regularizer + C * prediction_error
    
If we set $C$ large, it overwhelms any influence the regularizer has, and can lead to overfitting (minimizing prediction error on training set).

See also here: https://scikit-learn.org/0.15/auto_examples/svm/plot_rbf_parameters.html

### **1b**: Describe a setting of the hyperparameters `gamma` and `C` that is sure to *underfit* to any typical training set. 


To underfit, we should set `gamma` very small (like $10^{-6}$ or smaller). 

To underfit, we should also set $C$ very small, so that the regularization term has as large a possible influence as it can.

Why small $\gamma$? Given a fixed training point $x$, let's look at a nearby point $x+ \Delta$, where the pertubation between $x$ and the new point has magnitude $||\Delta||_2^2 = \sum_{f} \Delta_f^2 = \epsilon$

\begin{align}
k(x, x+\Delta) = e^{-\gamma ||(x+ \Delta) - x||_2^2} 
= e^{-\gamma \epsilon} \quad \approx e^{0} = 1 ~~~ \text{if gamma is small}
\end{align}

This will make nearby points have very similar predicted values, and thus lead to smoother predictions, smoother boundaries, and possible underfitting.


### **1c**: If you were performing grid search to find the SVM that generalized best to unseen data, what range of values for gamma, C would you recommend? Specify your answer with two lines of NumPy code (e.g. using a list like [1, 2, 3] or np.linspace or np.logspace).  Also provide 1-2 sentences of justification.

Assume that you can't afford more than 5 distinct values for each variable.

```
C_grid     = np.logspace(-4, +4, 5)   ### will use 10^-4, 10^-2, 10^0, 10^2, 10^4
gamma_grid = np.logspace(-2, +2, 5)    ### will use 10^-2, 10^-1, 10^0, 10^1, 10^2
```

Especially if the data is already normalized, both C and gamma might typically do well around values of 1 (=$10^0$), but could require logarithmic adjustments that make them both much smaller or much larger to do well (e.g. the RBF kernel may require very short or very long neighborhood lengthscales). 

There's a lot of wiggle room here, so as long as students tried some reasonable *log*-spaced grid with values above and below 1, most points were awarded.

### **1d**: Answer the following True/False questions, providing *one sentence* of explanation for each one

#### **1d(i)**: When used for binary classification, Support Vector Machines (SVMs) provide an estimate of the probability that a given feature vector $x_i$ will have a positive label: $p(Y_i = 1 | x_i)$.


FALSE. SVMs produce binary predictions by thresholding a real-valued score: $w^T x_i + b$

####  **1d(ii)**: An advantage of an SVM is that the optimal weight vector $w$ (a vector with one entry per feature) will typically be sparse. 


FALSE. SVM weight vectors $w$ are typically not sparse (see slides from lecture), but the learned $\alpha$ vector (one entry per example) in the kernelized version is often sparse.

#### **1d(iii)**: When choosing a kernel function $k$, it should be the case that for any finite dataset of $N$ distinct examples, the $N \times N$ kernel matrix is invertible.

FALSE. 

Kernel functions are required to be positive *semi-definite*, but that doesn't mean the K matrix is always invertible, even if there are N distinct points.

You can see some counter-examples (where N distinct points do NOT have an invertible $K$ matrix) in the attached script `when_is_kernel_matrix_invertible.py`

* linear kernel with $F > N$: typically $K$ is invertible
* linear kernel with $F < N$: typically $K$ is NOT invertible
* rbf kernel: typically invertible ($F >> N$, because this is an infinite feature space)

Invertibility would require **positive-definite** kernel function.

What does this mean for kernelized linear regression??

$$
K \alpha = y \rightarrow \alpha = K^{-1} y
$$

The simplification here is only valid if $K$ is invertible.

# Problem 2: Random Forests

### **2a:** Describe a setting of the hyperparameters `n_estimators`, `max_features`, and `min_samples_leaf` that is sure to *overfit* to any typical training set. You can assume no BAgging (`bootstrap=False`).


We can overfit with just one tree (n_estimators=1). You just let it use all the features (max_features=F) and allow it to grow until each leaf covers just one example (min_samples_leaf=1).

### **2b:** Describe a setting of the hyperparameters `n_estimators`, `max_features`, and `min_samples_leaf` that is sure to *underfit* to any typical training set. You can assume no BAgging (`bootstrap=False`).


We can underfit again with one tree, by just setting the minimum samples per leaf to a large value (perhaps equal to N, the total number of training examples). This will produce the same constant probability prediction for all examples.

#### **2c:** If you were performing grid search to find the RandomForest that generalized best to unseen data, what range of values for n_estimators, max_features, and min_samples_leaf would you recommend? Should we set bootstrap to True or False? Specify your answer with a few lines of NumPy code (e.g. using a list like [1, 2, 3] or np.linspace or np.logspace to set each variable). Include a sentence or two of justification. Assume that you can't afford more than 5 distinct values for each variable.


**Justification**: 

Bootstrap should always be turned on if we want to generalize well (training on separate samples is what helps RF hit the best bias-variance tradeoff).

Trying different minimum leaf sizes is likely the best key to managing overfitting.


## **2d:** Answer the following True/False questions, providing *one sentence* of explanation for each one:

###  **2d(i)**: When used for binary classification, RandomForests (RFs) can provide an estimate of the probability that a given feature vector $x_i$ will have a positive label: $p(Y_i = 1 | x_i)$.

TRUE. We can get probabilities from each decision tree, and just average them to get an estimated probability from the ensemble of many trees.

### **2d(ii)**: With bootstrap aggregating enabled, random forests will almost always severely overfit the training data if the number of trees used (e.g. the `n_estimators`) is very large (say, over 500).


FALSE. As the ISL textbook discusses, there isn't an overfitting danger of using too many trees.

### **2d(iii)**: When fitting random forests, it is generally a good idea to allow each node of each tree to consider as many features as possible (e.g. `max_features` should be large).


FALSE. The key idea of random forests is that we *decorrelate* between trees by making splits using different sets of features. Allowing all trees to greedily use the same features leads to little better performance than a single decision tree.

### **2d(iv)**: Random forests only use randomness when creating many similar datasets via BAgging. No other step of the algorithm uses randomness.


FALSE. Random forests can also randomly select subsets of features to use at each split.

# Problem 3

### **3a:** Can we apply ROC curves to binary classifiers that cannot easily produce probabilities $\hat{p}_i \in (0,1)$, but produce some real-valued scores $s_i \in \mathbb{R}$ for each example? How would we select the range of thresholds to evaluate?

Given the scores $s_n$ for each of the $N$ examples on the training set, we can use as all possible thresholds the unique values in $s = [s_1, \ldots s_N]$.

At each threshold $\tau$: we can do $\hat{y}_i = 1$ if $s_i \geq \tau$, and $\hat{y}_i = 0$ otherwise.


### **3b:** Suppose you fit a classifier to data, and you observe a TPR of 0.3 and an FPR of 0.7. Your friend says that this is worse than a random classifier (if plotted on an ROC curve, would fall below the TPR=FPR diagonal line), and so you should throw this result away. Is there anything better you can do? Describe how to tranform the predicted binary labels to reach better performance.  What TPR and FPR would you expect to achieve?



Given this situation, we could flip the binary value of each prediction.

That is, any prediction the original classifier made that was 0 would be flipped to a 1.
Any prediction from the original classifier that was a 1 would be a 0.

This would yield an FPR of 0.3 and a TPR of 0.7.

Think about the extreme case of a classifier with FPR=1.0 and TPR = 0.0. This is classifier that gets everything wrong. We should just make the opposite prediction for every example, and we'd get everything correct.